# Hello! Introduction to Agents from Scratch! 

# Agents Level 1 - Functions inside prompts

In [1]:
import os
import getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"var: ")

_set_env("OPENAI_API_KEY")

In [2]:
from openai import OpenAI
client = OpenAI()

def llm(sys_msg, prompt):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "system", "content": sys_msg},
                  {"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content


llm(sys_msg="You are a helpful assistant", prompt="What is the foundational idea of Agents based on LLMs?")

'The foundational idea of agents based on Large Language Models (LLMs) is to leverage the language understanding and generation capabilities of LLMs to create autonomous systems that can interact with users, process information, and perform tasks in a more human-like manner. Here are some key aspects of this foundational idea:\n\n1. **Natural Language Understanding**: Agents powered by LLMs can comprehend and interpret complex queries, requests, and instructions in natural language, making them accessible and user-friendly.\n\n2. **Contextual Awareness**: LLMs can maintain context over a conversation, allowing agents to follow up on prior exchanges and provide more coherent and relevant responses.\n\n3. **Task Execution**: These agents can be designed to execute a wide range of tasks, from providing information and answering questions to managing workflows and automating processes, based on user input.\n\n4. **Learning and Adaptation**: LLMs can analyze diverse inputs and user interact

In [3]:
def tool_create_file(file_name: str, content: str):
    with open(file_name, "w") as f:
        f.write(content)


tool_create_file("hello-world.md", "Hello, world!")

In [4]:
!cat hello-world.md

Hello, world!

**Experiment number 1**

Let's see if the model understands when to use the tool to create a file.

In [5]:
sys_msg_exp1 = "You are a helpful assistant, you have access ot a tool named `tool_create_file` that\
    can create a file given the file_name and the content. If you need to use this tool, your output should be\
    something like this: `tool_create_file(file_name, content)`"

prompt_exp1 = "Create a file called `llms-are-awesome.md` and inside write a 2 paragraph essay on the benefits and dangers of LLMs."

llm(sys_msg_exp1, prompt_exp1)

'tool_create_file("llms-are-awesome.md", "Large Language Models (LLMs) have revolutionized the way we interact with technology and access information. Their ability to generate human-like text has enabled a myriad of applications, from customer service automation to creative writing assistance. One of the key benefits of LLMs is their capacity to process and analyze vast amounts of data quickly, providing insights and solutions that might take humans significantly longer to develop. Additionally, LLMs can enhance productivity by performing repetitive tasks, freeing up human resources for more complex problem-solving and decision-making processes. Their versatility and adaptability make them increasingly valuable in various sectors, including education, healthcare, and entertainment.\\n\\nHowever, alongside their advantages, LLMs also pose certain dangers that must be addressed. The potential for misuse, such as generating misleading information or deepfake content, raises significant e

In [6]:
output = llm(sys_msg_exp1, prompt_exp1)
print(output)


tool_create_file("llms-are-awesome.md", "Large Language Models (LLMs) have revolutionized the way we interact with technology, offering unprecedented benefits across various fields. One of the key advantages of LLMs is their ability to process and generate human-like text, which facilitates improved communication and accessibility to information. They serve as powerful tools for content creation, customer support, and educational purposes, enabling users to receive instant assistance and generate ideas effortlessly. Additionally, LLMs can analyze vast amounts of data, uncovering insights that may not be immediately apparent to human analysts, thereby enhancing decision-making processes in business and research.

However, the powerful capabilities of LLMs also pose significant dangers if not managed responsibly. There are concerns about the potential for misinformation, as these models can produce convincing but factually incorrect outputs. This raises issues regarding trust and reliabi

In [13]:
tool_create_file("llms-are-awesome.md", "Large Language Models (LLMs) have revolutionized the way we interact with technology, offering unprecedented benefits across various fields. One of the key advantages of LLMs is their ability to process and generate human-like text, which facilitates improved communication and accessibility to information. They serve as powerful tools for content creation, customer support, and educational purposes, enabling users to receive instant assistance and generate ideas effortlessly. Additionally, LLMs can analyze vast amounts of data, uncovering insights that may not be immediately apparent to human analysts, thereby enhancing decision-making processes in business and research. However, the powerful capabilities of LLMs also pose significant dangers if not managed responsibly. There are concerns about the potential for misinformation, as these models can produce convincing but factually incorrect outputs. This raises issues regarding trust and reliability in information dissemination. Furthermore, the potential misuse of LLMs for generating harmful content, including hate speech or deepfakes, poses ethical challenges that society must confront. As we continue to integrate LLMs into our daily lives, it is crucial to develop guidelines and frameworks that mitigate these risks while maximizing the benefits that these advanced technologies can offer." )

# OpenAI Function Calling

In [15]:
from openai import OpenAI
from typing import List
import os
from pydantic import BaseModel, Field



client = OpenAI()

class FileCreation(BaseModel):
    file_name: str = Field(description="name of the file to be created")
    content: str = Field(description="The contents of the file to be created")
    


def llm_with_structured_outputs_for_file_creation(prompt):
    response = client.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[{"role": "system", "content": "You are a assistant that can create files for your user."},
                  {"role": "user", "content": prompt}],
        response_format=FileCreation
    )
    
    return response

prompt = "Create a file named lucas-loves-pancakes-a-bit-too-much-and-it-is-getting-weird-during-courses.md \
    Inside write a 2 paragraph essay on why would an AI insstructor love talking about \
    pancakes randomly during his courses."


response = llm_with_structured_outputs_for_file_creation(prompt)         

In [16]:
response

ParsedChatCompletion[FileCreation](id='chatcmpl-Aq02IlOICNhSiCZt7v7jLpUtUqaH9', choices=[ParsedChoice[FileCreation](finish_reason='stop', index=0, logprobs=None, message=ParsedChatCompletionMessage[FileCreation](content='{"file_name":"lucas-loves-pancakes-a-bit-too-much-and-it-is-getting-weird-during-courses.md","content":"In the realm of artificial intelligence, instructors often strive to make learning engaging and relatable. For an AI instructor, talking about pancakes can serve as an effective tool for capturing students\' attention and fostering a lighthearted atmosphere during potentially dense subjects. People generally love food, and pancakes, being a universally appreciated comfort food, can create a shared point of interest among diverse learners. This unexpected turn towards a familiar topic like pancakes can stimulate students’ interest, making them more receptive to complex concepts that might otherwise seem overly abstract or daunting.\\n\\nMoreover, discussing pancakes c

In [22]:
parsed_output = response.choices[0].message.parsed

file_name = parsed_output.file_name
content_of_file = parsed_output.content

In [23]:
file_name

'lucas-loves-pancakes-a-bit-too-much-and-it-is-getting-weird-during-courses.md'

In [24]:
content_of_file

"In the realm of artificial intelligence, instructors often strive to make learning engaging and relatable. For an AI instructor, talking about pancakes can serve as an effective tool for capturing students' attention and fostering a lighthearted atmosphere during potentially dense subjects. People generally love food, and pancakes, being a universally appreciated comfort food, can create a shared point of interest among diverse learners. This unexpected turn towards a familiar topic like pancakes can stimulate students’ interest, making them more receptive to complex concepts that might otherwise seem overly abstract or daunting.\n\nMoreover, discussing pancakes can function as a metaphor relevant to many fields of study. For instance, the process of making pancakes can be likened to structuring algorithms, where the right ingredients and precise measurements lead to a successful outcome. Emphasizing such analogies can help students visualize abstract ideas through tangible experience

In [25]:
tool_create_file(file_name=file_name, content=content_of_file)